<a href="https://colab.research.google.com/github/hamsterjoa/Kbank_playstore_review_crawler/blob/master/Kbank_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length{length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving review_dataset.csv to review_dataset (2).csv
User uploaded file "review_dataset.csv" with length553990 bytes


In [2]:
import pandas as pd
import io

import re
import nltk
from konlpy.tag import Okt; t=Okt()
from tqdm import tqdm
from pykospacing import Spacing
from collections import Counter

In [3]:
df = pd.read_csv(io.StringIO(uploaded['review_dataset.csv'].decode('utf-8')))
df = df.drop(['Unnamed: 0'], axis = 1)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220601,2022,6,1,1,동동,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠?? 범용공동인증서를 등록하려고...
1,2,20220626,2022,6,26,2,Younghoon Jeon,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요....
2,3,20220629,2022,6,29,1,Kuee Haeng Lee,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요. QR로 가져오기는 아예 안돼...
3,4,20220709,2022,7,9,3,brian lryan,갤럭시쓰는데 알림진동이 안 울립니다.(에티켓모드 아님) 전에 리뷰보니까 다른분들도 ...
4,5,20220625,2022,6,25,5,박태준,뭣같은 앱아이콘을 바꾼건 정말 좋네요. 거기다 묘하게 앱 자체의 속도도 빨라진 느낌...
...,...,...,...,...,...,...,...,...
2035,2036,20210506,2021,5,6,1,사람이 꽃보다 아름다워,갤럭시s9+사용 고객입니다.. 케이뱅크 앱이 실행이 안되고 로고만 뜨네요.. 3일내...
2036,2037,20210816,2021,8,16,1,Call Me K,저는 당신들이 너무 싫습니다. 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신...
2037,2038,20210223,2021,2,23,1,아주그냥,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중... 지친다
2038,2039,20210413,2021,4,13,1,zin lee,인증번호자체가 안오네요 삼성갤럭시 S9 최신OS 시간은 오후 8시부터 계속~ 쭉이요...


### 불용어 제거

In [4]:
text = ''
review = []
for each_line in df['content']:
  review.append(each_line)

In [5]:
def clean_str(text):
  pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '[^\w\s]'         # 특수기호제거
  text = re.sub(pattern=pattern, repl='', string=text)
  return text 

# 불용어 제거한 review 저장
review_ = []
for i in review:
  a = clean_str(i)
  review_.append(a)

In [6]:
print("Before Stopwords : ", review[239])
print("After Stopwords : ", review_[239])

Before Stopwords :  앱이 심심하면 안열리나봐요^^ 개짜증나네요 업데이트 안하면 안열리는거면 자동 업데이트로 이어지게 창이라도 띄웠으면 합니다. 업데이트는 툭하면 하는데 할때마다 영 열리지가 않으니 쓰라는건지말라는건지
After Stopwords :  앱이 심심하면 안열리나봐요 개짜증나네요 업데이트 안하면 안열리는거면 자동 업데이트로 이어지게 창이라도 띄웠으면 합니다 업데이트는 툭하면 하는데 할때마다 영 열리지가 않으니 쓰라는건지말라는건지


### 한글 표현만 남기기

In [7]:
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

In [8]:
# 특정 리뷰 전후 비교
print("Before Extraction : ",df['content'][135])
print("After Extraction : ", extract_word(df['content'][135]))

print("Before Extraction : ",df['content'][206])
print("After Extraction : ", extract_word(df['content'][206]))

Before Extraction :  알림시 진동안울림 !!!, 실행시 파란배경로고 나오기전 자주색+보라색 같은 배경이 먼저나옴 엄청 거슬림, 앱 전체가 너무 무거움 (기기 최신기종임)
After Extraction :  알림시 진동안울림      실행시 파란배경로고 나오기전 자주색 보라색 같은 배경이 먼저나옴 엄청 거슬림  앱 전체가 너무 무거움  기기 최신기종임 
Before Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림 ㅋㅋㅋㅋㅋㅋ개뿔 ㅋㅋㅋ 다시 고요해짐 ㅋㅋㅋ
After Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림       개뿔     다시 고요해짐    


In [9]:
# 전체 데이터에 extract_word 적용
df['content'] = df['content'].apply(lambda x:extract_word(x))

### 띄어쓰기 고치기

In [10]:
def no_space(text):
  text1 = re.sub('&nbsp;|&nbsp;|\n|\t|\r', '', text)
  text2 = re.sub('\n\n','', text1)
  return text2

In [11]:
spacing = Spacing()
print("Before Fixing : ",df['content'][123])
print("After Fixing : ", spacing(df['content'][123]))

print("Before Fixing : ",df['content'][258])
print("After Fixing : ", spacing(df['content'][258]))

Before Fixing :  앱딱열면첫페이지가갑자기보이지않아요 괜찮았는데어느순간안되서삭제후다시깔았는데도 첫페이지는하얗게나와요   계좌내역등을조회하려면마이페이지쪽으로가야만보여요  
After Fixing :  앱 딱 열면 첫 페이지가 갑자기 보이지 않아 요 괜찮았는데 어느 순간 안 되서 삭제 후 다시 깔았는데도 첫 페이지는 하얗게 나와 요 계좌 내역 등을 조회하려면 마이페이지 쪽으로 가야만 보여 요
Before Fixing :  케이뱅크켜면 얼마전부터 네트워크상태가 안좋다면서 다시깔아야해요 그리고는 초기로그인 딱한번만 제대로되요 데이터로도 와이파이로도 장소도바꿔봤지만 계속그러네요 업데이트때문인지 불편해요
After Fixing :  케이뱅크 켜면 얼마 전부터 네트워크 상태가 안 좋다면서 다시 깔아야 해요 그리고는 초기로그인 딱 한 번만 제대로 되요 데이터로도 와이파이로도 장소도 바꿔봤지만 계속 그러네요 업데이트 때문인지 불편해요


In [12]:
df['content'] = df['content'].apply(lambda x:no_space(x))

### 형태소 분석

In [13]:
a = len(review_)  # 불용어 제거한 리뷰
token_review_list=[]

for i in range(0,a):
  token_review = t.morphs(review_[i])
  token_review_list.append(token_review)

df['token_review'] = token_review_list

In [14]:
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content,token_review
0,1,20220601,2022,6,1,1,동동,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠 범용공동인증서를 등록하려고...,"[타, 기관, 인증서, 불러, 오기, 가, 안되는데, 대체, 무슨, 일이, 죠, 범..."
1,2,20220626,2022,6,26,2,Younghoon Jeon,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요 ...,"[다른건, 다, 모르겠고, 어플, 켰을, 때, 메인, 화면, 이, 뜨기까지, 시간,..."
2,3,20220629,2022,6,29,1,Kuee Haeng Lee,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요 로 가져오기는 아예 안돼...,"[타, 기관, 인증서, 를, 갱신, 하면서, 인증서, 가져오기를, 수행, 했는데요,..."
3,4,20220709,2022,7,9,3,brian lryan,갤럭시쓰는데 알림진동이 안 울립니다 에티켓모드 아님 전에 리뷰보니까 다른분들도 ...,"[갤럭시, 쓰는데, 알림, 진동, 이, 안, 울립니다, 에티켓, 모드, 아님, 전,..."
4,5,20220625,2022,6,25,5,박태준,뭣같은 앱아이콘을 바꾼건 정말 좋네요 거기다 묘하게 앱 자체의 속도도 빨라진 느낌...,"[뭣같, 은, 앱, 아이콘, 을, 바꾼건, 정말, 좋네요, 거기, 다, 묘, 하게,..."
...,...,...,...,...,...,...,...,...,...
2035,2036,20210506,2021,5,6,1,사람이 꽃보다 아름다워,갤럭시 사용 고객입니다 케이뱅크 앱이 실행이 안되고 로고만 뜨네요 일내...,"[갤럭시, s, 9, 사용, 고객, 입니다, 케이, 뱅크, 앱, 이, 실행, 이, ..."
2036,2037,20210816,2021,8,16,1,Call Me K,저는 당신들이 너무 싫습니다 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신...,"[저, 는, 당신, 들, 이, 너무, 싫습니다, 잘쓰고, 있던거를, 못, 쓰게, 하..."
2037,2038,20210223,2021,2,23,1,아주그냥,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중 지친다,"[접속, 을, 못, 하네, 인, 출, 좀, 할려고, 하니, 상담, 원, 은, 계속,..."
2038,2039,20210413,2021,4,13,1,zin lee,인증번호자체가 안오네요 삼성갤럭시 최신 시간은 오후 시부터 계속 쭉이요...,"[인증, 번호, 자체, 가, 안오네요, 삼성, 갤럭시, S, 9, 최신, OS, 시..."


In [15]:
df.to_csv('token_output.csv', encoding='utf-8-sig', mode='w')

### 평점별 단어 빈도분석

> 평점 5점인 리뷰에 대한 빈도분석



In [17]:
df_sorted_by_rating = df.sort_values(by='rating', ascending=False)

In [18]:
len(df_sorted_by_rating)

2040

In [19]:
# 평점 내림차순
df_sorted_by_rating

,id,date,dateYear,dateMonth,dateDay,rating,userName,content,token_review
2039,2040,20210403,2021,4,3,5,칼세이건,앱을설치하고 개설했는데 바탕화면에 앱이 안보여요,"[앱, 을, 설치, 하고, 개설, 했는데, 바탕화면, 에, 앱, 이, 안보, 여요]"
391,392,20220708,2022,7,8,5,이재성,편리함 실용적,"[편리함, 실용, 적]"
477,478,20210116,2021,1,16,5,김신석,원래 깔아 놓고는 잘 안썼는데 주변에서 케이뱅크 오픈뱅킹 편리하다고해서 써봤더니...,"[원래, 깔아, 놓고는, 잘, 안, 썼는데, 주변, 에서, 케이, 뱅크, 오픈, 뱅..."
468,469,20200701,2020,7,1,5,라울,한번도 불편한적없이 잘쓰고 있습니다 적금이율도 좋아서 계속 들고요 네이버케뱅카드...,"[한번, 도, 불편, 한적, 없이, 잘쓰고, 있습니다, 적금, 이, 율도, 좋아서,..."
451,452,20201230,2020,12,30,5,정다울,년넘게 급여이체포함하여 주거래로 쓰는중 심플하고 딱 있을것만 있고 불필요한 부...,"[2년, 넘게, 급여, 이체, 포함, 하, 여, 주, 거래, 로, 쓰는, 중, 심플..."
...,...,...,...,...,...,...,...,...,...
829,830,20210316,2021,3,16,1,달맞이꽃,그냥 구데기임 신분증 재확인 하래서 다시 할려고 사진 찍었는데 그 다음 페이지로 절...,"[그냥, 구, 데, 기, 임, 신분증, 재, 확인, 하래, 서, 다시, 할려고, 사..."
828,829,20190215,2019,2,15,1,Google 사용자,잘쓰다가 접속인 안되어서 제거 후 다시 설치했는데 바탕화면이 하얀색으로만 나오고 다...,"[잘쓰다가, 접속, 인, 안되어서, 제거, 후, 다시, 설치, 했는데, 바탕화면, ..."
826,827,20200924,2020,9,24,1,Jiwon Song,아 진짜 신분증촬영을 몇시간을 붙들고 쌩쑈를해도 어떻게 한번도 안찍힐수가있죠 ...,"[아, 진짜, 신분증, 촬영, 을, 몇, 시간, 을, 붙들고, 쌩쑈를해, 도, 어떻..."
824,825,20200624,2020,6,24,1,Changjun Jeon,멀쩡하게 잘 쓰고 있다가 오늘 업데이트 했더니 비정상적인 활동이라면서 로그인이 안...,"[멀쩡하게, 잘, 쓰고, 있다가, 오늘, 업데이트, 했더니, 비정상, 적, 인, 활..."


In [23]:
sorted_df = df_sorted_by_rating.groupby('rating').count()
sorted_df['content']

rating
1    1416
2     131
3     217
4      83
5     193
Name: content, dtype: int64

In [24]:
five_review = []
five_token_review = []

for i in range(0, 2040):
  if(df['rating'][i] == 5):
    five_review.append(df['content'][i]) # 5점 리뷰 모음, len=193
    five_token_review.append(df['token_review'][i])

In [25]:
len(five_token_review)

193

In [26]:
# 단어 빈도수 계산
frequency = {}

for token_review in five_token_review:
  for word in token_review:
    count = frequency.get(word,0)
    frequency[word] = count + 1

In [30]:
sort = sorted(frequency.items(), key=lambda x: x[1], reverse=True)
sort

[('이', 138),
 ('도', 90),
 ('가', 72),
 ('에', 69),
 ('뱅크', 63),
 ('을', 60),
 ('잘', 55),
 ('사용', 53),
 ('케이', 49),
 ('앱', 47),
 ('은행', 45),
 ('으로', 42),
 ('로', 36),
 ('하고', 35),
 ('너무', 35),
 ('좋아요', 34),
 ('은', 33),
 ('에서', 29),
 ('를', 26),
 ('어플', 25),
 ('입니다', 24),
 ('계좌', 24),
 ('요', 24),
 ('수수료', 24),
 ('이체', 23),
 ('더', 22),
 ('는', 22),
 ('이용', 21),
 ('들', 21),
 ('안', 20),
 ('있습니다', 19),
 ('하기', 18),
 ('뱅킹', 18),
 ('중', 17),
 ('인증', 17),
 ('다시', 17),
 ('것', 16),
 ('깔끔하고', 16),
 ('가입', 16),
 ('편하고', 16),
 ('수', 16),
 ('할', 16),
 ('적', 15),
 ('제', 14),
 ('같아요', 14),
 ('합니다', 14),
 ('오픈', 14),
 ('만', 14),
 ('의', 13),
 ('하는', 13),
 ('인식', 13),
 ('보다', 13),
 ('설치', 13),
 ('못', 13),
 ('적금', 13),
 ('많이', 12),
 ('다른', 12),
 ('했는데', 12),
 ('때', 12),
 ('카드', 12),
 ('인데', 12),
 ('업데이트', 12),
 ('한번', 12),
 ('하', 12),
 ('대출', 12),
 ('좀', 12),
 ('정말', 11),
 ('다', 11),
 ('최고', 11),
 ('시', 11),
 ('재', 11),
 ('매우', 11),
 ('거래', 11),
 ('고', 11),
 ('해도', 11),
 ('접속', 11),
 ('버전', 11),
 ('쓰고', 11),
 ('

> 평점 1점인 리뷰에 대한 빈도분석

In [35]:
one_review = []
one_token_review = []

for i in range(0, 2040):
  if(df['rating'][i] == 1):
    one_review.append(df['content'][i]) # 1점 리뷰 모음, len=1416
    one_token_review.append(df['token_review'][i])

# 단어 빈도수 계산
frequency = {}

for token_review in five_token_review:
  for word in token_review:
    count = frequency.get(word,0)
    frequency[word] = count + 1

sort = sorted(frequency.items(), key=lambda x: x[1], reverse=True)

In [36]:
sort

[('이', 1549),
 ('도', 749),
 ('을', 666),
 ('앱', 648),
 ('가', 615),
 ('에', 610),
 ('안', 522),
 ('하고', 393),
 ('뱅크', 389),
 ('인증', 380),
 ('어플', 369),
 ('은', 349),
 ('못', 327),
 ('로', 321),
 ('에서', 316),
 ('으로', 313),
 ('다시', 312),
 ('만', 310),
 ('계좌', 309),
 ('를', 307),
 ('은행', 275),
 ('는', 274),
 ('케이', 271),
 ('신분증', 256),
 ('사용', 247),
 ('화면', 245),
 ('요', 241),
 ('계속', 240),
 ('왜', 232),
 ('다', 230),
 ('해도', 229),
 ('설치', 209),
 ('가입', 207),
 ('재', 203),
 ('인식', 201),
 ('한', 200),
 ('접속', 196),
 ('다른', 185),
 ('잘', 181),
 ('이체', 180),
 ('실행', 178),
 ('확인', 177),
 ('안되고', 171),
 ('하', 165),
 ('오류', 164),
 ('할', 163),
 ('들', 161),
 ('너무', 157),
 ('삭제', 154),
 ('좀', 147),
 ('진짜', 147),
 ('로그인', 142),
 ('거', 137),
 ('업데이트', 137),
 ('인증서', 137),
 ('뭐', 137),
 ('등록', 135),
 ('고', 134),
 ('하는데', 134),
 ('중', 131),
 ('고객', 131),
 ('후', 131),
 ('번호', 130),
 ('개설', 129),
 ('해서', 128),
 ('건지', 127),
 ('문제', 125),
 ('촬영', 122),
 ('했는데', 122),
 ('하는', 119),
 ('부터', 119),
 ('입니다', 118),
 ('것', 115